In [31]:
import numpy as np
from keras.layers import GRU,Dense
from keras.models import Sequential
from tensorflow.keras.callbacks import LambdaCallback,ModelCheckpoint,ReduceLROnPlateau


In [21]:
with open('poems.txt','r',encoding='utf-8') as file:
    text=file.read()

In [22]:
characters=sorted(set(text))

char_index=dict((c,i) for i,c in enumerate(characters))
index_char=dict((i,c) for i,c in enumerate(characters))

In [23]:
seq_len=40
step_size=3

input_seq=[]
target_seq=[]

for i in range(0,len(text)-seq_len,step_size):
    input_seq.append(text[i:i+seq_len])
    target_seq.append(text[i+seq_len])

x=np.zeros((len(input_seq),seq_len,len(characters)),dtype=bool)
y=np.zeros((len(input_seq),len(characters)),dtype=bool)

for i,sent in enumerate(input_seq):
    for t, char in enumerate(sent):
        x[i,t,char_index[char]]=1
    y[i, char_index[target_seq[i]]] = 1

In [27]:
model=Sequential()
model.add(GRU(128,input_shape=(seq_len,len(characters))))
model.add(Dense(len(characters),activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='rmsprop')
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_1 (GRU)                 (None, 128)               69504     
                                                                 
 dense_1 (Dense)             (None, 51)                6579      
                                                                 
Total params: 76083 (297.20 KB)
Trainable params: 76083 (297.20 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [28]:
def sample(preds,temp=1.0):
  preds=np.asarray(preds).astype('float64')
  preds=np.log(preds)/temp
  exp_pred=np.exp(preds)
  preds=exp_pred/np.sum(exp_pred)
  probs=np.random.multinomial(1,preds,1)
  return np.argmax(probs)


In [34]:
filepath='wts.hdf5'
checkpoint=ModelCheckpoint(filepath,monitor='loss',save_best_only=True,mode='min')

reduce_a=ReduceLROnPlateau(monitor='loss',factor=0.2,patience=1)

In [56]:
def gen_text(start_text='He clasps the',length=50,temp=1.0):
  model.load_weights('wts.hdf5')
  start_text='He clasps the'
  generated=start_text
  for _ in range(100):
    x_pred=np.zeros((1,seq_len,len(characters)))
    for t,char in enumerate(start_text):
      x_pred[0,t,char_index[char]]=1

    pred=model.predict(x_pred,verbose=0)[0]
    next_index=sample(pred,temp)
    next_char=index_char[next_index]

    generated+=next_char
    start_text=start_text[1:]+next_char
    print(generated)
  return generated
print_callback = LambdaCallback(on_epoch_end = gen_text)

In [58]:
callbacks=[checkpoint,reduce_a,print_callback]

model.fit(x, y, batch_size = 128, epochs = 3, callbacks = callbacks)

Epoch 1/3
5/5 [==============================] - ETA: 0s - loss: 2.9547He clasps thee
He clasps theeh
He clasps theehr
He clasps theehrr
He clasps theehrrt
He clasps theehrrtt
He clasps theehrrttt
He clasps theehrrttt 
He clasps theehrrttt d
He clasps theehrrttt d 
He clasps theehrrttt d a
He clasps theehrrttt d ad
He clasps theehrrttt d ado
He clasps theehrrttt d adoa
He clasps theehrrttt d adoa 
He clasps theehrrttt d adoa l
He clasps theehrrttt d adoa le
He clasps theehrrttt d adoa let
He clasps theehrrttt d adoa letn
He clasps theehrrttt d adoa letnw
He clasps theehrrttt d adoa letnw 
He clasps theehrrttt d adoa letnw e
He clasps theehrrttt d adoa letnw ee
He clasps theehrrttt d adoa letnw eet
He clasps theehrrttt d adoa letnw eety
He clasps theehrrttt d adoa letnw eetye
He clasps theehrrttt d adoa letnw eetyet
He clasps theehrrttt d adoa letnw eetyete
He clasps theehrrttt d adoa letnw eetyeteg
He clasps theehrrttt d adoa letnw eetyetegf
He clasps theehrrttt d adoa letnw eetyetegf 